# Amazon Bedrock を AWS SDK for Python (boto3) から使用する基本的なサンプル

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** このノートブックは、SageMaker Studioの **Data Science 3.0** カーネルで動作します

### モデルの情報の取得

In [ ]:
import boto3
import pprint

bedrock = boto3.client("bedrock", region_name="us-east-1")
pprint.pprint(bedrock.list_foundation_models()["modelSummaries"])
print("-" * 80)
pprint.pprint(bedrock.get_foundation_model(modelIdentifier = "amazon.titan-text-express-v1"))

### テキスト生成
参考：[モデル呼び出し API を使用して Amazon Bedrock で Meta Llama 3 を呼び出す | Amazon Bedrockユーザーガイド](https://docs.aws.amazon.com/ja_jp/bedrock/latest/userguide/bedrock-runtime_example_bedrock-runtime_InvokeModel_MetaLlama3_section.html)

In [ ]:
import boto3
import json

bedrock = boto3.client('bedrock-runtime')

accept = 'application/json'
contentType = 'application/json'

modelId = "meta.llama3-8b-instruct-v1:0"

prompt = "日本の首都はどこですか？簡潔に日本語で回答してください。"

formatted_prompt = f"""
<|begin_of_text|>
<|start_header_id|>user<|end_header_id|>
{prompt}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""


# リクエストBODYの指定
body = json.dumps({
    "prompt": formatted_prompt,
    "temperature": 0.5,
    "top_p": 0.1,
    "max_gen_len": 100
})


response = bedrock.invoke_model(body=body,
                                modelId=modelId,
                                accept=accept, 
                                contentType=contentType)

# API レスポンスから BODY を取り出す
response_body = json.loads(response.get('body').read())

# レスポンスBODYから応答テキストを取り出す

#print(response_body)

outputText = response_body.get('generation')


print(outputText)



### テキスト要約
参考：[モデル呼び出し API を使用して Amazon Bedrock で Meta Llama 3 を呼び出す | Amazon Bedrockユーザーガイド](https://docs.aws.amazon.com/ja_jp/bedrock/latest/userguide/bedrock-runtime_example_bedrock-runtime_InvokeModel_MetaLlama3_section.html)

In [ ]:
import boto3
import json

boto3_bedrock = boto3.client("bedrock-runtime")

prompt = """

下記のテキストの要約を日本語で提示して下さい。
<text>
AWS はお客様からのフィードバックをすべて取り入れ、本日 Amazon Bedrock を発表できることを嬉しく思います。\
AI21 Labs、Anthropic、Stability AI、Amazon の FM に API 経由でアクセスできるようにする新しいサービスです。 \
Bedrock は、顧客が FM を使用して生成 AI ベースのアプリケーションを構築および拡張する最も簡単な方法です。\
すべての建設業者のアクセスを民主化します。 Bedrock は、さまざまな強力な FM にアクセスする機能を提供します。
テキストと画像用 -  同じく発表している 2 つの新しい LLM で構成される Amazons Titan FM を含む \
今日、スケーラブルで信頼性が高く安全な AWS マネージド サービスを通じて。 Bedrock のサーバーレス エクスペリエンスにより、\
顧客は、やろうとしていることに適したモデルを簡単に見つけて、非公開ですぐに開始できます \
FM を独自のデータでカスタマイズし、AWS を使用してアプリケーションに簡単に統合してデプロイできます。\
インフラストラクチャ を管理する必要がなく、使い慣れたツールや機能を利用できます。\
(統合を含むさまざまなモデルをテストするための実験や、大規模な FM を管理するためのパイプラインなどの Amazon SageMaker ML 機能を使用します)。
</text>
"""

formatted_prompt = f"""
<|begin_of_text|>
<|start_header_id|>user<|end_header_id|>
{prompt}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

body = json.dumps({"prompt": formatted_prompt,
                 "max_gen_len": 512,
                 "temperature":0.5,
                 "top_p":0.5
                  }) 
                  

modelId = "meta.llama3-8b-instruct-v1:0"
accept = 'application/json'
contentType = 'application/json'

response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

outputText = response_body.get('generation')


print(outputText)



### ストリーム出力
参考：[レスポンスストリームで Invoke Model API を使用して Amazon Bedrock で Meta Llama 3 を呼び出す | Amazon Bedrockユーザーガイド](https://docs.aws.amazon.com/ja_jp/bedrock/latest/userguide/bedrock-runtime_example_bedrock-runtime_InvokeModelWithResponseStream_MetaLlama3_section.html)

In [ ]:
import boto3
import json

bedrock_runtime = boto3.client('bedrock-runtime')

prompt = """
AWS とは何かを簡単に日本語で説明してください。
"""

formatted_prompt = f"""
<|begin_of_text|>
<|start_header_id|>user<|end_header_id|>
{prompt}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

body = json.dumps({"prompt": formatted_prompt,
                 "max_gen_len": 512,
                 "temperature":0.5,
                 "top_p":0.5
                  }) 


modelId = "meta.llama3-8b-instruct-v1:0"
accept = "application/json"
contentType = "application/json"

response = bedrock_runtime.invoke_model_with_response_stream(
        body=body, modelId=modelId, accept=accept, contentType=contentType
)

# 1. Extract and print the response text with number
# stream = response.get('body')
# count = 0
# if stream:
#     for event in stream:
#         chunk = event.get('chunk')
#         if chunk:
#            chunk_bytes = chunk.get('bytes').decode()
#            result = json.loads(chunk_bytes)
#            count +=1
#            print(f'[{count}] {result["generation"]}')
            

# 2. Extract and print the response text in real-time.
for event in response["body"]:
    chunk = json.loads(event["chunk"]["bytes"])
    if "generation" in chunk:
        print(chunk["generation"], end="")


### 課題
下記のサンプルと同じプロンプトを使用して、Mistral Large (24.02) モデルに対して invokeModel を発行するコードを完成させてみよう

#### Amazon Titan Text G1 Express のサンプル

In [ ]:
import boto3
import json
from botocore.exceptions import ClientError


#
# Amazon Titan Text G1 Express
#
print("---" * 42)
print("Amazon Titan Text G1 Express")
print("---" * 42)


try:
    bedrock_runtime = boto3.client('bedrock-runtime')
    prompt_data = """Describe the purpose of a 'hello world' program in one line."""
    config = {"maxTokenCount": 512, "temperature": 0.5, "topP": 0.9 }

    body = json.dumps(
            {
                "inputText": prompt_data, "textGenerationConfig": config
            }
    )
    modelId = "amazon.titan-text-express-v1"
    accept = "application/json"
    contentType = "application/json"

    response = bedrock_runtime.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    print(response_body.get("results")[0].get("outputText"))

except ClientError as error:
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error


#### Meta Llama 3 8B Instruct のサンプル

In [ ]:
import boto3
import json
from botocore.exceptions import ClientError

#
# Meta Llama 3 8B Instruct
#
print("---" * 42)
print("Meta Llama 3 8B Instruct")
print("---" * 42)


try:
    bedrock_runtime = boto3.client('bedrock-runtime')
    prompt_data = """Describe the purpose of a 'hello world' program in one line."""
    max_gen_len = 512
    temperature = 0.5
    top_p = 0.9

    body =json.dumps(
        {
            "prompt": prompt_data, "max_gen_len": max_gen_len, "temperature": temperature, "top_p": top_p
        }
    )
    modelId = "meta.llama3-8b-instruct-v1:0"
    accept = "application/json"
    contentType = "application/json"

    response = bedrock_runtime.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    print(response_body["generation"])

except ClientError as error:
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error


#### Mistral Large (24.02) を使用するコードを下のセルに記述してみましょう

* ヒント 1： AWS マネジメントコンソールの Amazon Bedrock のページの左側メニューで「プロバイダー」を選択し、Mistral AI のタブから使用するモデルの API のパラメータを確認しましょう
* ヒント 2: レスポンスの受け取り方については、[このドキュメント](https://docs.aws.amazon.com/ja_jp/bedrock/latest/userguide/bedrock-runtime_example_bedrock-runtime_InvokeModel_MistralAi_section.html)　を参考にしましょう　  